<a href="https://colab.research.google.com/github/hasnatosman/mini_projects/blob/main/Smart_Library_Management_System.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Project Overview: Smart Library Management System

**Core Python features:** variables, loops, conditionals, functions
Object-Oriented Programming (OOP)
Data structures: lists, dictionaries, sets
File handling
Exception handling
Modules and libraries
Database integration
Multi-threading
Networking (basic client-server)
GUI development (optional)
Deployment options (CLI, GUI, or web)

## Objective
The system manages a library's books and members, supporting tasks like:

Adding/removing books.
Issuing/returning books.
Managing member details.
Viewing transaction history.
Search functionality for books/members.
Multithreaded notifications for due dates.
Optional GUI for ease of use.

# Step 1: Database Setup

In [9]:
import sqlite3
import threading
import time

# Initialize the database
def initialize_db():
    conn = sqlite3.connect('db.sqlite')
    cursor = conn.cursor()

    cursor.execute('''
        CREATE TABLE IF NOT EXISTS books (
            id INTEGER PRIMARY KEY,
            title TEXT,
            author TEXT,
            copies INTEGER
        )
    ''')

    cursor.execute('''
        CREATE TABLE IF NOT EXISTS members (
            id INTEGER PRIMARY KEY,
            name TEXT,
            email TEXT,
            borrowed_books TEXT
        )
    ''')

    conn.commit()
    conn.close()

## Explanation:
*  `books` table stores details about each book.
*  `members` table tracks member details and books they’ve borrowed.
*  `borrowed_books` is stored as a string of comma-separated book IDs.

# Step 2: Book Management

In [10]:
# Book management
class Book:
    def __init__(self, title, author, copies):
        self.title = title
        self.author = author
        self.copies = copies

    def add_to_db(self):
        conn = sqlite3.connect('db.sqlite')
        cursor = conn.cursor()
        cursor.execute('''
            INSERT INTO books (title, author, copies)
            VALUES (?, ?, ?)
        ''', (self.title, self.author, self.copies))
        conn.commit()
        conn.close()
        print(f"Book '{self.title}' added successfully!")

    @staticmethod
    def list_books():
        conn = sqlite3.connect('db.sqlite')
        cursor = conn.cursor()
        cursor.execute('SELECT * FROM books')
        books = cursor.fetchall()
        conn.close()

        print("\nAvailable Books:")
        for book in books:
            print(f"ID: {book[0]}, Title: {book[1]}, Author: {book[2]}, Copies: {book[3]}")

## Explanation:
*  `add_to_db`: Adds a book to the `books` table.
*  `list_books`: Displays all books.

# Step 3: Member Management

In [11]:
# Member management
class Member:
    def __init__(self, name, email):
        self.name = name
        self.email = email

    def register(self):
        conn = sqlite3.connect('db.sqlite')
        cursor = conn.cursor()
        cursor.execute('''
            INSERT INTO members (name, email, borrowed_books)
            VALUES (?, ?, ?)
        ''', (self.name, self.email, ""))
        conn.commit()
        conn.close()
        print(f"Member '{self.name}' registered successfully!")

    @staticmethod
    def list_members():
        conn = sqlite3.connect('db.sqlite')
        cursor = conn.cursor()
        cursor.execute('SELECT * FROM members')
        members = cursor.fetchall()
        conn.close()

        print("\nRegistered Members:")
        for member in members:
            print(f"ID: {member[0]}, Name: {member[1]}, Email: {member[2]}, Borrowed Books: {member[3]}")

## Explanation:
*  `register`: Adds a new member to the `members` table.
*  `list_members`: Displays all members.

# Step 4: Issue/Return Books

In [12]:
# Issuing and returning books
def issue_book(member_id, book_id):
    conn = sqlite3.connect('db.sqlite')
    cursor = conn.cursor()

    cursor.execute('SELECT copies FROM books WHERE id = ?', (book_id,))
    book = cursor.fetchone()
    if not book or book[0] <= 0:
        print("Book not available!")
        return

    cursor.execute('UPDATE books SET copies = copies - 1 WHERE id = ?', (book_id,))
    cursor.execute('SELECT borrowed_books FROM members WHERE id = ?', (member_id,))
    borrowed_books = cursor.fetchone()[0]
    updated_books = f"{borrowed_books},{book_id}" if borrowed_books else str(book_id)
    cursor.execute('UPDATE members SET borrowed_books = ? WHERE id = ?', (updated_books, member_id))

    conn.commit()
    conn.close()
    print("Book issued successfully!")


def return_book(member_id, book_id):
    conn = sqlite3.connect('db.sqlite')
    cursor = conn.cursor()

    cursor.execute('UPDATE books SET copies = copies + 1 WHERE id = ?', (book_id,))
    cursor.execute('SELECT borrowed_books FROM members WHERE id = ?', (member_id,))
    borrowed_books = cursor.fetchone()[0].split(',')
    updated_books = ','.join([book for book in borrowed_books if book != str(book_id)])
    cursor.execute('UPDATE members SET borrowed_books = ? WHERE id = ?', (updated_books, member_id))

    conn.commit()
    conn.close()
    print("Book returned successfully!")

## Explanation:
*  `issue_book`:Decreases the number of available copies of a book.Updates the member's borrowed book list.
*  `return_book`:Increases the number of available copies.Removes the book from the member's borrowed book list.

# Step 5: Notifications with Multi-threading

In [23]:
from datetime import datetime, timedelta

def notify_due_books():
    while True:
        conn = sqlite3.connect('db.sqlite')
        cursor = conn.cursor()

        print("Checking for due books...")

        # Simulated logic for fetching due books
        # Assuming a 'due_date' column in the 'members' table to track return deadlines
        cursor.execute('''
            SELECT members.name, members.email, members.borrowed_books
            FROM members
            WHERE borrowed_books IS NOT NULL AND borrowed_books != ''
        ''')
        members = cursor.fetchall()

        for member in members:
            name, email, borrowed_books = member
            borrowed_books_list = borrowed_books.split(',')

            for book_id in borrowed_books_list:
                # Example: Assuming a due date 7 days after issuing
                # Adjust logic based on how you implement issuing/return dates
                issued_date = datetime.now() - timedelta(days=7)  # Simulated issued date
                due_date = issued_date + timedelta(days=7)

                if due_date < datetime.now():
                    print(f"Reminder: Member {name} ({email}) has an overdue book with ID {book_id}.")

        conn.close()
        time.sleep(10)  # Wait 10 seconds before the next check


Checking for due books...


# Step 6: CLI (Optional)

In [25]:
# Main CLI
def main():
    initialize_db()

    # Start notification thread
    notification_thread = threading.Thread(target=notify_due_books, daemon=True)
    notification_thread.start()

    while True:
        print("\n--- Smart Library Management System ---")
        print("1. Add Book")
        print("2. List Books")
        print("3. Register Member")
        print("4. List Members")
        print("5. Issue Book")
        print("6. Return Book")
        print("7. Exit")
        choice = input("Enter your choice: ")

        if choice == '1':
            title = input("Enter book title: ")
            author = input("Enter book author: ")
            copies = int(input("Enter number of copies: "))
            book = Book(title, author, copies)
            book.add_to_db()

        elif choice == '2':
            Book.list_books()

        elif choice == '3':
            name = input("Enter member name: ")
            email = input("Enter member email: ")
            member = Member(name, email)
            member.register()

        elif choice == '4':
            Member.list_members()

        elif choice == '5':
            member_id = int(input("Enter member ID: "))
            book_id = int(input("Enter book ID: "))
            issue_book(member_id, book_id)

        elif choice == '6':
            member_id = int(input("Enter member ID: "))
            book_id = int(input("Enter book ID: "))
            return_book(member_id, book_id)

        elif choice == '7':
            print("Exiting the system. Goodbye!")
            break

        else:
            print("Invalid choice! Please try again.")


if __name__ == '__main__':
    main()

Checking for due books...

--- Smart Library Management System ---
1. Add Book
2. List Books
3. Register Member
4. List Members
5. Issue Book
6. Return Book
7. Exit
Checking for due books...
Checking for due books...
Checking for due books...
Checking for due books...
Checking for due books...
Checking for due books...
Checking for due books...
Checking for due books...
Checking for due books...
Checking for due books...
Checking for due books...
Checking for due books...
Checking for due books...
Checking for due books...
Checking for due books...
Checking for due books...
Checking for due books...
Checking for due books...
Checking for due books...
Checking for due books...
Checking for due books...
Checking for due books...
Checking for due books...
Checking for due books...
Checking for due books...
Checking for due books...
Checking for due books...
Checking for due books...
Checking for due books...
Checking for due books...
Checking for due books...
Checking for due books...
Che

KeyboardInterrupt: Interrupted by user